In [14]:
from copy import deepcopy
from collections import defaultdict
import random

## Reading and initializing functions

In [15]:
def get_states(filename, sudokusize=9):
    '''
    get_states is to read multiple sudokus in the format:
    ..8..4..5...64..5 etc.
    '''
    sudokus = []
    with open(filename) as sudoku_states:
        for state in sudoku_states:
            sudoku = []
            row = 1
            for i, content in enumerate(state):
                if i%sudokusize == 0 and i > 0:
                    row+=1
                try:
                    int(content)
                    clause = int(str(row) + str(i%sudokusize+1) + content)
                    sudoku.append(clause)
                except:
                    # is not convertible to int
                    continue
            sudokus.append(sudoku)
    return sudokus

def get_state_from_dimacs(filename, sudokusize=9):
    '''
    This function reads a sudoku state in dimacs format
    '''
    state = []
    with open(filename) as f:
        for clause in f:
            for literal in clause.split():
                if literal!= '0':  
                    state.append(int(literal))
    return state

def get_rules(filename):
    constraints = {} # dictionary with constraints as: clausenumber->[literals]
    pointers = {} # dictionary with pointers to clause numbers for literals as: literals->[clausenumbers]
    with open(filename) as sudoku_rules:
        for index, clause in enumerate(sudoku_rules):
            if index == 0:
                continue # skip first line
            constraints[index]=[]
            for literal in clause.split():
                lit = int(literal)
                if lit!=0:
                    constraints[index].append(lit)
                    if lit not in pointers.keys():
                        pointers[lit]=[index]
                    else:
                        pointers[lit].append(index)
                    
    return constraints, pointers

## DP helper functions for constraints updates

In [16]:
def assign_literal(constraints, pointers, literal, assignments):
    '''
    assign the given literal to be True.
    assign_literal is aimed to be one recursion stack, so deepcopies are made
    in order to properly backtrack
    '''
    constraintsCopied = deepcopy(constraints)
    pointersCopied = deepcopy(pointers)
    assignmentsCopied = deepcopy(assignments)
    assignmentsUpdated = update_assigments(assignmentsCopied, literal)
    constraintsUpdated, pointersUpdated = update_constraints(constraintsCopied, pointersCopied, literal)
    return constraintsUpdated, pointersUpdated, assignmentsUpdated
     
def update_constraints(constraints, pointers, literal):
    
    ## Find clauses with literal and remove them
    clauses = list(pointers[literal])
    for clause in clauses:
        for lit in constraints[clause]:
            pointers[lit].remove(clause)
        del constraints[clause]
    del pointers[literal]
    
    ## Find clauses with counter-literal and remove its counter-literal
    counterLiteral = -literal
    clauses = list(pointers[counterLiteral])
    del pointers[counterLiteral]
    for clause in clauses:
        constraints[clause].remove(counterLiteral)

    return constraints, pointers

def update_assigments(assignments, literal):
    if literal in assignments:
        print("already assigned", literal, "(in this stack), returning...")
        return
    else:
        assignments.append(literal)
        return assignments
    
def check_tautologies(constraints, pointers):
    foundOne = False
    for clause in constraints.keys():
        literals = constraints[clause]
        counterLiterals = [-x for x in literals]
        for i in literals:
            for j in counterLiterals:
                if i == j:
                    del constraints[clause]
                    pointers[i].remove(clause)
                    foundOne = True
                    break
    if foundOne:
        print("Found tautologies and removed corresponding clauses")
    else:
        print("No tautologies found")
                  
    return constraints, pointers  

## DP cases

In [21]:
def empty_clause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
        
    return False

def unit_clause(constraints): 
    for clause in constraints.keys():
        if len(constraints[clause])==1:
            literal = constraints[clause][0]
            return literal

    return False

def unit_propagate(constraints, pointers, assignments):
    literal = unit_clause(constraints)
    while(not empty_clause(constraints) and literal):
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
        literal = unit_clause(constraints)
    return constraints, pointers, assignments

def pure_literal(literals):
    for literal in literals:
        if -literal not in literals:
            return literal
        
    return False

def random_literal(literals): 
    literal = random.choice(literals) 
    return literal

## DP main function

In [18]:
def DP(constraints, pointers, assignments): 
    
    constraints, pointers, assignments = unit_propagate(constraints, pointers, assignments)
    
    if len(constraints)==0:
        return assignments
   
    if empty_clause(constraints):
        return False
    
    all_literals = list(pointers.keys())
    
    literal = pure_literal(all_literals)
    if literal:
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
        DP(constraints, pointers, assignments)
    
    literal = random_literal(all_literals)
    constraintsUpdated, pointersUpdated, assignmentsUpdated = assign_literal(constraints, pointers, literal, assignments)
    assignmentsFinal = DP(constraintsUpdated, pointersUpdated, assignmentsUpdated)
    if assignmentsFinal:
        return assignmentsFinal
    else:
        constraints, pointers, assignments = assign_literal(constraints, pointers, -literal, assignments)
        return DP(constraints, pointers, assignments)

In [20]:
def set_up():
    
    ## Get board and rules
#     state = get_state_from_dimacs('sudoku-example.txt')
    state = get_states('1000 sudokus.txt')[5]
    print("Sudoku initial board state", state)
    constraints, pointers = get_rules('sudoku-rules.txt')
    print("Length of clauses:", len(constraints))
    
    ## Update constraints according to initial board state
    initialAssignments = []
    for literal in state:
        constraints, pointers, initialAssignments = assign_literal(constraints, pointers, literal, initialAssignments)
    print("clauses after board state:", len(constraints.keys()))

    ## Check tautologies
    constraints, pointers = check_tautologies(constraints, pointers)
    
    ## Run DP
    print("Starting DP algorithm...")
    assignments = DP(constraints, pointers, initialAssignments)
    if assignments:
        print("Solution found")
        boardPositions = [a for a in assignments if a > 0]
        print(boardPositions)
    else:
        print("No solution found")
        
set_up()

Sudoku initial board state [165, 222, 264, 281, 323, 358, 382, 468, 474, 518, 546, 629, 651, 677, 695, 736, 819, 825, 863, 886, 933, 991]
Length of clauses: 11988
clauses after board state: 11900
No tautologies found
Starting DP algorithm...
Solution found
[165, 222, 264, 281, 323, 358, 382, 468, 474, 518, 546, 629, 651, 677, 695, 736, 819, 825, 863, 886, 933, 991, 688, 616, 413, 126, 643, 662, 554, 634, 535, 432, 571, 421, 527, 569, 583, 592, 489, 496, 314, 215, 375, 276, 366, 956, 967, 761, 852, 831, 878, 897, 717, 912, 772, 793, 979, 844, 111, 341, 142, 173, 253, 187, 194, 399, 298, 159, 247, 138, 239, 337, 445, 457, 948, 755, 749, 784, 985, 728, 924]
